## To change the Location graphs to the cytoscape format

In [1]:
##Import section
import json
from pprint import pprint
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.drawing import *


In [2]:
CONNECTIVITY_PATH  = "../src/assets/connectivity/"
GRAPH_PATH  = "../src/assets/graph/"
SCAN_NAME = "2t7WUuJeko7"
FILE_EXT = "_connectivity.json"

In [3]:
def load_nav_graphs(scans):
    ''' Load connectivity graph for each scan '''

    def distance(pose1, pose2):
        ''' Euclidean distance between two graph poses '''
        return ((pose1['pose'][3]-pose2['pose'][3])**2\
          + (pose1['pose'][7]-pose2['pose'][7])**2\
          + (pose1['pose'][11]-pose2['pose'][11])**2)**0.5

    graphs = {}
    for scan in scans:
        with open('../src/assets/connectivity/%s_connectivity.json' % scan) as f:
            G = nx.Graph()
            positions = {}
            data = json.load(f)
            for i,item in enumerate(data):
                if item['included']:
                    for j,conn in enumerate(item['unobstructed']):
                        if conn and data[j]['included']:
                            positions[item['image_id']] = np.array([item['pose'][3], 
                                    item['pose'][7], item['pose'][11]]);
                            assert data[j]['unobstructed'][i], 'Graph should be undirected'
                            G.add_edge(item['image_id'],data[j]['image_id'],weight=distance(item,data[j]))
            nx.set_node_attributes(G, values=positions, name='position')
            graphs[scan] = G
    return graphs

In [4]:
def convert2cytoscapeJSON(G, out_path, min_x, min_y):
    # load all nodes into nodes array
    final = {}
    final["nodes"] = []
    final["edges"] = [] 
    positions = nx.get_node_attributes(G,'position')
    #print(positions)
    for node in G.nodes():
        new_node = {}
        new_node["data"] = {}
        new_node["data"]["id"] = node
        new_node["data"]["label"] = node
        new_node["position"] = {}
        new_node["position"]["x"] = (positions[node][0] + min_x) * 100
        new_node["position"]["y"] = (positions[node][1] + min_y) * 100
        new_node["grabbable"] = False
        new_node["events"] = {}
        new_node["events"]["click"] = "yes"
        
        #new_node["locked"] = True
        final["nodes"].append(new_node.copy())
    #load all edges to edges array
    for edge in G.edges():
        new_edge = {}
        new_edge["data"]={}
        new_edge["data"]["id"]=edge[0]+edge[1]
        new_edge["data"]["source"]=edge[0]
        new_edge["data"]["target"]=edge[1]
        final["edges"].append(new_edge)
        
    with open(out_path, "w") as f:
        json.dump(final, f)
    return json.dumps(final)


In [5]:
def convert2visJSON(G, out_path, min_x, min_y):
    # load all nodes into nodes array
    final = {}
    final["nodes"] = []
    final["edges"] = [] 
    positions = nx.get_node_attributes(G,'position')
    #print(positions)
    for i,node in enumerate(G.nodes()):
        new_node = {}
        new_node["id"] = node
        new_node["label"] = "Node " + str(i) 
        new_node["x"] = (positions[node][0] + min_x) * 100
        new_node["y"] = (positions[node][1] + min_y) * 100
        final["nodes"].append(new_node.copy())
    #load all edges to edges array
    for edge in G.edges():
        new_edge = {}
        new_edge["from"]=edge[0]
        new_edge["to"]=edge[1]
        final["edges"].append(new_edge)
        
    with open(out_path, "w") as f:
        json.dump(final, f)
    return json.dumps(final)

In [6]:
with open(CONNECTIVITY_PATH + "scans.txt") as f:
    scans = []
    for line in f:
        scans.append(line.strip())
        
        
graphs = load_nav_graphs(scans)

for scan in scans:
    positions = nx.get_node_attributes(graphs[scan],'position')
    min_x = min([pos[0] for pos in positions.values()]) + 1
    min_y = min([pos[0] for pos in positions.values()]) + 1
    convert2visJSON(graphs[scan], GRAPH_PATH +  scan+ '_connectivity.json', min_x, min_y)
